In [ ]:
pip install nltk

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import nltk
import itertools
import re

Filtering of label and prediction into tokens

Given a sentence of label and a sentence of prediction, compare them such that the metric indicates accuracy of the generated text being accurate to description. Since this is medical image captioning, importance of sentence can be captured in major description words and medical terminology, such as body parts and diagnosis.

1. Filter both label and prediction of stopwords (unneccesary content)
2. Extract important words from the label
3. Check for presence of important words in the prediction
4. Check loss based on that (currently 0 1 loss of existance)

In [ ]:
nltk.download('stopwords', quiet=True)

class Tokenize:
  def __init__(self):
    self._stopwords = list(itertools.islice(stopwords.words('english'), 100)) #grab first 100 stopwords
    self._medical = [] #load in medical dictionary

  def filter(self, caption: str):
    words = re.findall(r'\b\w+\b', caption.lower())
    return [word for word in words if word not in self._stopwords]

  def extract(self, tokenized_caption: str):
    '''Clean the text of stopwords and return important words'''
    medical = []
    important = []

    # for word in words:
    #   if word.lower in medical_dictionary:
    #     me
    return list(set(medical + important))

Loss metric

In [24]:
#loss methods
def loss(Filter, original, prediction):
  og = set(Filter.filter(original))
  pred = set(Filter.filter(prediction))

  common_words = og.intersection(pred)

  return len(common_words) / len(og)

Example Usage

In [28]:
filter = Tokenize()
label = "computed tomography scan of the chest showing a large mass in the right lower lobe of the left lower lobe."
prediction = "xray shows a hairline fracture in lower extremity"

print(loss(filter, label, prediction))

#information retrieval for data parsing in order to create accurate evaluation metric


0.09090909090909091


TODO:

develop better metric/research other metrics

medical dictionary, ngrams, synonyms, ordering

ensure better runtime of retrieval